<a href="https://colab.research.google.com/github/st20080675/dreambooth/blob/main/dreambooth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##This is my notebook of testing dreambooth
#reference:

[YouTube videl](https://www.youtube.com/watch?v=TKTBZ5zNxT0)

[hugging face](https://huggingface.co/docs/diffusers/training/dreambooth#finetuning-with-prior-preserving-loss)

A company [Aragon](https://www.aragon.ai/) seems to build their product based on this model alone. :


In [ ]:
# !pip install git+https://github.com/huggingface/diffusers
!git clone https://github.com/huggingface/diffusers
%cd diffusers/
!pip install -e .
!pip install -U -r ./examples/dreambooth/requirements.txt
# !pip install xformers
# !pip install accelerate
!accelerate config default
!pip install bitsandbytes

Cloning into 'diffusers'...
remote: Enumerating objects: 48462, done.
remote: Counting objects: 100% (435/435), done.
remote: Compressing objects: 100% (227/227), done.
remote: Total 48462 (delta 252), reused 317 (delta 188), pack-reused 48027
Receiving objects: 100% (48462/48462), 30.46 MiB | 15.84 MiB/s, done.
Resolving deltas: 100% (35719/35719), done.
/content/diffusers/diffusers
Obtaining file:///content/diffusers/diffusers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for diffusers (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.24.0.dev0-0.editable-py3-none-any.whl size=11009 sha256=04ca4f141f1ab07004edef5c8a8afdf89d054c7c16cdfb95f9fa92b9613bac20
  Stored in directory: /tmp/pip-ephem-wheel-cache-yzcbl8qz/wheels/e2/39/95/cefa614da96e8b1ec3f962f86d335e1936336ad256d5a263b8


Configuration already exists at /root/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.


## notes


*   I need to use `bitsandbytes` with `--use_8bit_adam` option, otherwise output memory.  see [hugging face doc](https://huggingface.co/docs/diffusers/training/dreambooth#16gb-gpu)
*   mount drive for data access notes: `export OUTPUT_DIR="path_to_saved_model"` does not work, `output_dir=$OUTPUT_DIR` the output_dir end up empty. I need to use `%env OUTPUT_DIR=/content/output/`



In [ ]:
%env MODEL_NAME=runwayml/stable-diffusion-v1-5
%env INSTANCE_DIR=/content/drive/MyDrive/data/dreambooth_test/crystal_liu/
%env OUTPUT_DIR=/content/drive/MyDrive/data/dreambooth_test/output/
%env CLASS_DIR=/content/drive/MyDrive/data/dreambooth_test/woman/

# !accelerate launch ./examples/dreambooth/train_dreambooth.py \
#   --pretrained_model_name_or_path=$MODEL_NAME  \
#   --instance_data_dir=$INSTANCE_DIR \
#   --output_dir=$OUTPUT_DIR \
#   --instance_prompt="a photo of sks woman" \
#   --resolution=512 \
#   --train_batch_size=2 \
#   --gradient_accumulation_steps=1 \
#   --use_8bit_adam \
#   --learning_rate=1e-6 \
#   --lr_scheduler="constant" \
#   --lr_warmup_steps=0 \
#   --max_train_steps=800 \

!accelerate launch ./examples/dreambooth/train_dreambooth.py \
    --pretrained_model_name_or_path=$MODEL_NAME  \
    --instance_data_dir=$INSTANCE_DIR \
    --output_dir=$OUTPUT_DIR \
    --class_data_dir=$CLASS_DIR \
    --with_prior_preservation --prior_loss_weight=1.0 \
    --instance_prompt="photo of sks woman" \
    --class_prompt="photo of woman" \
    --resolution=512 \
    --train_batch_size=1 \
    --gradient_accumulation_steps=1 --gradient_checkpointing \
    --use_8bit_adam \
    --mixed_precision="fp16" \
    --learning_rate=1e-6 \
    --lr_scheduler="constant" \
    --lr_warmup_steps=0 \
    --num_class_images=500 \
    --max_train_steps=1000

env: MODEL_NAME=runwayml/stable-diffusion-v1-5
env: INSTANCE_DIR=/content/drive/MyDrive/data/dreambooth_test/crystal_liu/
env: OUTPUT_DIR=/content/drive/MyDrive/data/dreambooth_test/output/
env: CLASS_DIR=/content/drive/MyDrive/data/dreambooth_test/woman/
2023-11-17 19:31:46.306439: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-17 19:31:46.306496: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-17 19:31:46.306530: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-17 19:31:48.412388: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could 

##inference
# notes
I need to reinstall diffusers, otherwise got: `ImportError: cannot import name 'StableDiffusionPipeline' from 'diffusers'`

In [ ]:
!pip uninstall diffusers -y
!pip install diffusers

Found existing installation: diffusers 0.24.0.dev0
Uninstalling diffusers-0.24.0.dev0:
  Successfully uninstalled diffusers-0.24.0.dev0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.8 MB/s eta 0:00:00


In [ ]:
from diffusers import StableDiffusionPipeline, DiffusionPipeline, UNet2DConditionModel
from transformers import CLIPTextModel
import torch

# Load the pipeline with the same arguments (model, revision) that were used for training
# model_id = "CompVis/stable-diffusion-v1-4"
model_id = "runwayml/stable-diffusion-v1-5"

unet = UNet2DConditionModel.from_pretrained("/content/drive/MyDrive/data/dreambooth_test/output/unet")

# if you have trained with `--args.train_text_encoder` make sure to also load the text encoder
text_encoder = CLIPTextModel.from_pretrained("/content/drive/MyDrive/data/dreambooth_test/output/text_encoder")

pipeline = DiffusionPipeline.from_pretrained(
    model_id, unet=unet, text_encoder=text_encoder, dtype=torch.float16, use_safetensors=True
)
pipeline.to("cuda")

# Perform inference, or save, or push to the hub
prompt = "A photo of sks woman"
image = pipeline(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]

image.save("/content/drive/MyDrive/data/dreambooth_test/output/liu-garden.png")
# pipeline.save_pretrained("dreambooth-pipeline")

# ----------------------------------------------------------------

# prompt = "A photo of zwx woman in a bucket"
# image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]

# image.save("liu-bucket.png")

# # -----------------------------------------------------------------------------------
# from diffusers import StableDiffusionPipeline
# model_id = "outputs/1000"
# pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

# example_prompts = [
# "close-up photo of zwx person in a suit",
# "closeup portrait of zwx person in a suit, highly detailed, with future city skyline in the background",
# "Professional headshot of zwx person inside a modern office building, highly detailed, business, open eyes",
# "LinkedIn profile picture of zwx person, close up, professional, businessman, blue suit, 8k"
# ]

# num_images = 10
# for i in range(num_images):
#     prompt = example_prompts[random.randint(0, len(example_prompts) - 1)]
#     image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]
#     image.save(f"profile_{i}.png")

Keyword arguments {'dtype': torch.float16} are not expected by StableDiffusionPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


  0%|          | 0/50 [00:00<?, ?it/s]